In [18]:
import Functions as Func
import AdvancedFuncs as af


import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from xgboost.callback import EarlyStopping

import optuna

Here we outline the process to perform our XGBoost hyperparameter tuning

The main motivation for our approach is the difficulty tuning XGBoost using cross validation, due to its ability to quickly overfit to validation data that it is allowed to iterate on. This required a method which did not allow the model to keep using the same dataset repeatedly, so it could not infer its properties and overfit to it as highly flexible models are capable of.

XGBoost also has a large number of sensitive parameters to tune, which motivated our decision to use a mixed random grid search-Bayesian optimization approach which we implemented with Optuna. This approach is very quick to find effective hyperparameter candidates, but it can also make overfitting worse if the objective is set improperly.

We avoided the overfitting problem by using a boostrapping approach, where both training data and validation data are selected from boostrapped samples, for each iteration. This means that our model never seems the same dataset more than once, which largely eliminates the problem of overfitting as the model is incentivized to learn the overall trends of the data rather than fit to noise.

In [19]:
# We first go through our normal data pre-processing steps

df = Func.load_and_basic_process()
df_X, df_Y = Func.df_data_process(df)

X_model, X_test, Y_model, Y_test = train_test_split(df_X, df_Y, test_size=0.1, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_model, Y_model, test_size=0.2, random_state=2)

In [28]:
def objective(trial):
    
    # We define a hyperparameter space to optimize. We could expand this to additional parameters or expand the range of existing ones, if desired

    n = trial.suggest_int('n', 400, 900)
    params = {
        'objective': trial.suggest_categorical('objective', ['count:poisson']),
        'eval_metric': 'mae',
        "tree_method": "hist",
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.4, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'max_delta_step': trial.suggest_float('max_delta_step', 0, 3),
        'min_child_weight': trial.suggest_float('min_child_weight', 0, 10),
        'gamma': trial.suggest_float('gamma', 0, 2)
    }
    
    # We define boostrap training/validation sets from the original model data for each loop
    X_Train_boot, Y_Train_boot, X_Val_boot, Y_Val_boot = af.generate_train_val_sets(X_model, Y_model)
    
    # We convert the boostrapped dataframes into efficient XGBoost DMatrix format
    dtrain_reg = xgb.DMatrix(X_Train_boot, label=Y_Train_boot["Frequency"], weight=Y_Train_boot["Exposure"])
    dval_reg = xgb.DMatrix(X_Val_boot, label=Y_Val_boot["Frequency"], weight=Y_Val_boot["Exposure"])
    

    evals = [(dtrain_reg, 'train'), (dval_reg, 'eval')]

    # Here we train our model on our current set of hyperparameters
    early_stopping_cb = EarlyStopping(rounds=10, save_best=True)
    model = xgb.train(
        params=params,
        dtrain=dtrain_reg,
        num_boost_round=n,
        evals=evals,
        callbacks=[early_stopping_cb],
    )

    # Generate predictions and calculate MAE
    best_iteration = model.best_iteration
    preds = model.predict(dval_reg, iteration_range=(0, best_iteration + 1))
    mae = mean_absolute_error(Y_Val_boot["Frequency"], preds, sample_weight=Y_Val_boot["Exposure"])

    # Our optimizer will try to minimize the MAE returned 
    return mae

In [29]:
# We initialize our experiment here and set the number of trials
# Our output is verbose so that we can see the stability of the XGBoost within its internal validation process on training data in addition to our own validation process

trial_num = 50

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=trial_num)  

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-02-26 15:40:12,211] A new study created in memory with name: no-name-cf6147bb-69a7-476f-8d02-cabbbb4a9c81


[0]	train-mae:0.87019	eval-mae:0.87077
[1]	train-mae:0.85537	eval-mae:0.85594
[2]	train-mae:0.84085	eval-mae:0.84141
[3]	train-mae:0.82663	eval-mae:0.82717
[4]	train-mae:0.81269	eval-mae:0.81321
[5]	train-mae:0.79903	eval-mae:0.79954
[6]	train-mae:0.78565	eval-mae:0.78615
[7]	train-mae:0.77254	eval-mae:0.77303
[8]	train-mae:0.75970	eval-mae:0.76017
[9]	train-mae:0.74711	eval-mae:0.74757
[10]	train-mae:0.73478	eval-mae:0.73523
[11]	train-mae:0.72270	eval-mae:0.72313
[12]	train-mae:0.71086	eval-mae:0.71128
[13]	train-mae:0.69926	eval-mae:0.69967
[14]	train-mae:0.68789	eval-mae:0.68830
[15]	train-mae:0.67676	eval-mae:0.67715
[16]	train-mae:0.66585	eval-mae:0.66623
[17]	train-mae:0.65516	eval-mae:0.65552
[18]	train-mae:0.64468	eval-mae:0.64504
[19]	train-mae:0.63442	eval-mae:0.63476
[20]	train-mae:0.62437	eval-mae:0.62470
[21]	train-mae:0.61451	eval-mae:0.61483
[22]	train-mae:0.60486	eval-mae:0.60517
[23]	train-mae:0.59540	eval-mae:0.59570
[24]	train-mae:0.58614	eval-mae:0.58642
[25]	train

[I 2024-02-26 15:40:48,375] Trial 0 finished with value: 0.13689653732652016 and parameters: {'n': 774, 'objective': 'count:poisson', 'learning_rate': 0.1764613836876073, 'max_depth': 7, 'max_delta_step': 2.1570598056070276, 'min_child_weight': 6.7018341822924965, 'gamma': 2.211098762414819}. Best is trial 0 with value: 0.13689653732652016.


[0]	train-mae:0.97157	eval-mae:0.97227
[1]	train-mae:0.97076	eval-mae:0.97145
[2]	train-mae:0.96994	eval-mae:0.97064
[3]	train-mae:0.96913	eval-mae:0.96982
[4]	train-mae:0.96832	eval-mae:0.96901
[5]	train-mae:0.96751	eval-mae:0.96820
[6]	train-mae:0.96669	eval-mae:0.96738
[7]	train-mae:0.96588	eval-mae:0.96657
[8]	train-mae:0.96507	eval-mae:0.96576
[9]	train-mae:0.96427	eval-mae:0.96495
[10]	train-mae:0.96346	eval-mae:0.96414
[11]	train-mae:0.96265	eval-mae:0.96333
[12]	train-mae:0.96184	eval-mae:0.96253
[13]	train-mae:0.96104	eval-mae:0.96172
[14]	train-mae:0.96023	eval-mae:0.96091
[15]	train-mae:0.95943	eval-mae:0.96011
[16]	train-mae:0.95862	eval-mae:0.95930
[17]	train-mae:0.95782	eval-mae:0.95850
[18]	train-mae:0.95702	eval-mae:0.95770
[19]	train-mae:0.95622	eval-mae:0.95690
[20]	train-mae:0.95542	eval-mae:0.95609
[21]	train-mae:0.95462	eval-mae:0.95529
[22]	train-mae:0.95382	eval-mae:0.95449
[23]	train-mae:0.95302	eval-mae:0.95369
[24]	train-mae:0.95222	eval-mae:0.95290
[25]	train

[I 2024-02-26 15:41:30,133] Trial 1 finished with value: 0.5096326633422836 and parameters: {'n': 827, 'objective': 'count:poisson', 'learning_rate': 0.1800195081147199, 'max_depth': 8, 'max_delta_step': 5.213547305404442, 'min_child_weight': 5.316807687103214, 'gamma': 0.4886800600790403}. Best is trial 0 with value: 0.13689653732652016.


...

[0]	train-mae:0.96595	eval-mae:0.96677
[1]	train-mae:0.96487	eval-mae:0.96569
[2]	train-mae:0.96380	eval-mae:0.96462
[3]	train-mae:0.96273	eval-mae:0.96355
[4]	train-mae:0.96165	eval-mae:0.96248
[5]	train-mae:0.96059	eval-mae:0.96141
[6]	train-mae:0.95952	eval-mae:0.96034
[7]	train-mae:0.95845	eval-mae:0.95928
[8]	train-mae:0.95739	eval-mae:0.95821
[9]	train-mae:0.95632	eval-mae:0.95715
[10]	train-mae:0.95526	eval-mae:0.95609
[11]	train-mae:0.95420	eval-mae:0.95503
[12]	train-mae:0.95314	eval-mae:0.95397
[13]	train-mae:0.95208	eval-mae:0.95291
[14]	train-mae:0.95102	eval-mae:0.95186
[15]	train-mae:0.94997	eval-mae:0.95080
[16]	train-mae:0.94891	eval-mae:0.94975
[17]	train-mae:0.94786	eval-mae:0.94870
[18]	train-mae:0.94681	eval-mae:0.94765
[19]	train-mae:0.94576	eval-mae:0.94660
[20]	train-mae:0.94471	eval-mae:0.94555
[21]	train-mae:0.94366	eval-mae:0.94450
[22]	train-mae:0.94261	eval-mae:0.94346
[23]	train-mae:0.94157	eval-mae:0.94241
[24]	train-mae:0.94053	eval-mae:0.94137
[25]	train

[I 2024-02-26 15:53:41,539] Trial 29 finished with value: 0.43769197572651153 and parameters: {'n': 786, 'objective': 'count:poisson', 'learning_rate': 0.10983942496767726, 'max_depth': 6, 'max_delta_step': 4.438051123899365, 'min_child_weight': 3.0635040400627878, 'gamma': 1.9681510248047729}. Best is trial 10 with value: 0.13582984126313355.


[0]	train-mae:0.89255	eval-mae:0.89240
[1]	train-mae:0.88340	eval-mae:0.88326
[2]	train-mae:0.87435	eval-mae:0.87422
[3]	train-mae:0.86541	eval-mae:0.86530
[4]	train-mae:0.85658	eval-mae:0.85649
[5]	train-mae:0.84786	eval-mae:0.84778
[6]	train-mae:0.83924	eval-mae:0.83917
[7]	train-mae:0.83073	eval-mae:0.83068
[8]	train-mae:0.82232	eval-mae:0.82228
[9]	train-mae:0.81401	eval-mae:0.81399
[10]	train-mae:0.80580	eval-mae:0.80579
[11]	train-mae:0.79770	eval-mae:0.79770
[12]	train-mae:0.78969	eval-mae:0.78970
[13]	train-mae:0.78177	eval-mae:0.78180
[14]	train-mae:0.77396	eval-mae:0.77400
[15]	train-mae:0.76623	eval-mae:0.76629
[16]	train-mae:0.75861	eval-mae:0.75867
[17]	train-mae:0.75107	eval-mae:0.75115
[18]	train-mae:0.74362	eval-mae:0.74372
[19]	train-mae:0.73627	eval-mae:0.73638
[20]	train-mae:0.72900	eval-mae:0.72912
[21]	train-mae:0.72183	eval-mae:0.72196
[22]	train-mae:0.71474	eval-mae:0.71488
[23]	train-mae:0.70773	eval-mae:0.70788
[24]	train-mae:0.70081	eval-mae:0.70097
[25]	train

[W 2024-02-26 15:53:57,191] Trial 30 failed with parameters: {'n': 438, 'objective': 'count:poisson', 'learning_rate': 0.1288850336738635, 'max_depth': 10, 'max_delta_step': 2.3588198051585256, 'min_child_weight': 1.994859179218643, 'gamma': 3.9613628674711157} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Kyleg\anaconda3\envs\TabNetImp\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Kyleg\AppData\Local\Temp\ipykernel_15112\2487825304.py", line 31, in objective
    model = xgb.train(
            ^^^^^^^^^^
  File "c:\Users\Kyleg\anaconda3\envs\TabNetImp\Lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\Kyleg\anaconda3\envs\TabNetImp\Lib\site-packages\xgboost\training.py", line 181, in train
    bst.update(dtrain, i, obj)
  File "c:\Users\Kyleg\anac

KeyboardInterrupt: 